In [1]:
pip install -U Flask-SQLAlchemy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install flask-login

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install bcrypt

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install PyJWT==1.6.4

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from flask import Flask, request, redirect, session, jsonify, make_response
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from datetime import datetime
from flask_bcrypt import Bcrypt
import jwt
import datetime
from functools import wraps


app = Flask(__name__)
db = SQLAlchemy(app)
bcrypt = Bcrypt(app)
app.config['SECRET_KEY'] = 'abc'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'


class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(15), unique=True)
    password = db.Column(db.String(80))
                         
                    
class Topic(db.Model):
    __tablename__ = 'topic'
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(200))
    subtitle = db.Column(db.String(200))
    user_id = db.Column(db.Integer)
    advertise = db.Column(db.Boolean)
    
db.create_all()

def token_required(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        token = None

        if 'x-access-token' in request.headers:
            token = request.headers['x-access-token']

        if not token:
            return jsonify({'message' : 'Token is missing!'}), 401

        try: 
            data = jwt.decode(token, app.config['SECRET_KEY'])
            current_user = User.query.filter_by(id=data['id']).first()
        except:
            return jsonify({'message' : 'Token is invalid!'}), 401

        return f(current_user, *args, **kwargs)

    return decorated

@app.route('/signup', methods=['POST'])
def register():
    data = request.get_json()
    hashed_password = bcrypt.generate_password_hash(data['password'])
    new_user = User(username=data['username'], password=hashed_password)
    db.session.add(new_user)
    db.session.commit()
    return jsonify({'message' : 'Registration completed!'})


@app.route('/login')
def login():
    auth = request.authorization

    if not auth or not auth.username or not auth.password:
        return make_response('Could not verify', 401, {'WWW-Authenticate' : 'Basic realm="Login required!"'})

    user = User.query.filter_by(username=auth.username).first()

    if not user:
        return make_response('Could not verify', 401, {'WWW-Authenticate' : 'Basic realm="Login required!"'})

    if bcrypt.check_password_hash(user.password, auth.password):
        token = jwt.encode({'id' : user.id, 'exp' : datetime.datetime.utcnow() + datetime.timedelta(minutes=15)}, app.config['SECRET_KEY'])

        return jsonify({'token' : token.decode('UTF-8')})

    return make_response('Could not verify', 401, {'WWW-Authenticate' : 'Basic realm="Login required!"'})

@app.route('/topic', methods=['GET'])
@token_required

def get_all_topics(current_user):
    topics = Topic.query.all()

    output = []

    for topic in topics:
        topic_data = {}
        topic_data['advertise'] = topic.advertise
        topic_data['id'] = topic.id
        topic_data['subtitle'] = topic.subtitle
        topic_data['title'] = topic.title
        topic_data['user_id'] = topic.user_id
        output.append(topic_data)

    return jsonify({'topics' : output})


@app.route('/mytopic', methods=['GET'])
@token_required
def get_topics_of_one_user(current_user):
    topics = Topic.query.filter_by(user_id=current_user.id).all()

    output = []

    for topic in topics:
        topic_data = {}
        topic_data['advertise'] = topic.advertise
        topic_data['id'] = topic.id
        topic_data['subtitle'] = topic.subtitle
        topic_data['title'] = topic.title
        topic_data['user_id'] = topic.user_id
        output.append(topic_data)

    return jsonify({'topics' : output})


@app.route('/topic/<topic_id>', methods=['GET'])
@token_required
def get_one_topic(current_user, topic_id):
    topic = Topic.query.filter_by(id=topic_id, user_id=current_user.id).first()

    if not topic:
        return jsonify({'message' : 'No topic found'})

    topic_data = {}
    topic_data['advertise'] = topic.advertise
    topic_data['id'] = topic.id
    topic_data['subtitle'] = topic.subtitle
    topic_data['title'] = topic.title
    topic_data['user_id'] = topic.user_id

    return jsonify(topic_data)

@app.route('/topic', methods=['POST'])
@token_required
def create_topic(current_user):
    data = request.get_json()

    new_topic = Topic(title=data['title'], advertise=False, subtitle=data['subtitle'], user_id=current_user.id)
    db.session.add(new_topic)
    db.session.commit()

    return jsonify({'message' : "Topic created!"})

@app.route('/topic/<topic_id>', methods=['PUT'])
@token_required
def update_topic(current_user, topic_id):
    topic = Topic.query.filter_by(id=topic_id, user_id=current_user.id).first()

    if not topic:
        return jsonify({'message' : 'No topic found!'})

    topic.advertise = True
    db.session.commit()

    return jsonify({'message' : 'Topic item has been updated!'})

@app.route('/topic/<topic_id>', methods=['DELETE'])
@token_required
def delete_topic(current_user, topic_id):
    topic = Topic.query.filter_by(id=topic_id, user_id=current_user.id).first()

    if not topic:
        return jsonify({'message' : 'No topic found!'})

    db.session.delete(topic)
    db.session.commit()

    return jsonify({'message' : 'Topic deleted!'})


if __name__ == "__main__":
    app.run('localhost', port=9041)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/helenm/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:9041/ (Press CTRL+C to quit)
